In this notebook we create an example of a simple chatbot that searches the web in order to respond.  
This is only meant as an example, and is not meant to be a state-of-the-art chatbot.

In [1]:
%load_ext autoreload
%autoreload 2
import guidance
from guidance import models, gen, select, substring, string, prefix_tree, regex, user, assistant, system

2023-11-20 22:28:22.684472: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-20 22:28:22.748580: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
llama2 = models.LlamaCppChat("/home/marcotcr_google_com/work/models/llama-2-13b-chat.Q6_K.gguf", n_gpu_layers=-1, n_ctx=4096)

### Code for calling a search engine (Bing)

You need to set up a bing api project (it's free), and change the api_key file path below

In [3]:
import os
import diskcache
import pathlib
import requests
import html
from urllib.parse import urlparse
import urllib.parse
import io
import html
import html.parser

curr_dir = './'
_bing_cache = diskcache.Cache(f"{curr_dir}/../bing.diskcache")

with open(os.path.expanduser('/home/marcotcr_google_com/.bing_api_key'), 'r') as file:
    subscription_key = file.read().replace('\n', '')

class MLStripper(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = io.StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def bing_search(search_terms, count=10):
    if type(search_terms) == str:
        search_terms = [search_terms]
    search_url = "https://api.bing.microsoft.com/v7.0/search"

    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    search_results = []
    for search_term in search_terms:
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "cout": count}
        params_key = search_term + "-___-" + str(count)
        if params_key not in _bing_cache or "webPages" not in _bing_cache[params_key]:
            response = requests.get(search_url, headers=headers, params=params)
            response.raise_for_status()
            _bing_cache[params_key] = response.json()
        data = _bing_cache[params_key]["webPages"]["value"]
        for r in data:
            r["snippet_text"] = strip_tags(r["snippet"])
        search_results.extend(data)
    return search_results
def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['snippet_text']} for x in results]

In [5]:
def format_snippets(snippets, start=1):
    ret = ''
    for i, s in enumerate(snippets, start=start):
        title = s['title']
        snippet = s['snippet']
        ret += f'[{i}] {title}\n'
        ret += f'{snippet}\n\n'
    return ret
        
print(format_snippets(top_snippets('current us president')))

[1] Joe Biden: The President | The White House
Joe Biden The President House — with thousands of train rides in between Download Official Portrait President Biden represented Delaware for 36 years in the U.S. Senate before becoming the...

[2] President of the United States - Wikipedia
The president of the United States ( POTUS) [A] is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces . The power of the presidency has grown substantially [11] since the first president, George ...

[3] List of presidents of the United States - Wikipedia
Grover Cleveland served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to the discrepancy between the number of presidencies and the number of persons who have served as president. [5] The incumbent president is Joe Biden. [6]




## Guidance code

In [6]:
from guidance import gen, select, silent, capture, Tool, one_or_more, any_char, commit_point
import collections

First, let's write an initial prompt with some few-shot examples of REACT.  
This prompt is trying to get the model to:
1. Search the web to find answers to questions
2. Read snippets, extract quotes that are relevant (to minimize hallucination)
3. Do extra searches if need
4. Print all of the quotes it gathered (from all search queries)
5. Answer

In [7]:
@guidance
def init_system(lm):
    # silent makes this whole thing not appear in the jupyter notebook visualization
    with silent():
        with system():
            lm += '''\
            You are a nice chatbot that answers users' queries. Whenever you get a question, you run a loop of [Thought, Act, Observation] until you hit a response, where Act is one of (search, respond, extract_quotes, list_quotes).
            You have a storehouse of facts, but you *always* search if you need information that depends on current events, since what you know may be out-of-date.
            After searching, if you find information in the snippets, you should extract quotes.
            If you need, you can make multiple searches.
            You should only ask the user for more information is their question is ambiguous.
            If the snippets are ambiguous or don't contain enough information, you should make additional searches instead of asking the user.
            Here are some example interactions:
            ---
            User: Who is the current president of the US?
            Thought: Since the president may have changed, my information may be out of date. Thus, I need to search the web for this.
            Act: search(current US president)
            Observation: 
            [1] Joe Biden: The President | The White House
            Joe Biden The President House — with thousands of train rides in between Download Official Portrait President Biden represented Delaware for 36 years in the U.S. Senate before becoming the...

            [2] President of the United States - Wikipedia
            The president of the United States ( POTUS) [A] is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces . The power of the presidency has grown substantially [11] since the first president, George ...

            [3] List of presidents of the United States - Wikipedia
            Grover Cleveland served two non-consecutive terms and is therefore counted as the 22nd and 24th president of the United States, giving rise to the discrepancy between the number of presidencies and the number of persons who have served as president. [5] The incumbent president is Joe Biden. [6]
            
            Thought: Snippets [1] and [3] clearly indicate that Joe Biden is the current president.
            Act: extract_quotes()
            Question: Who is the current president of the US?
            Relevant quotes from the snippets:
            - Snippet 1: "Joe Biden: The President | White House"
            - Snippet 3: "The incumbent president is Joe Biden"
            Thought: I have enough information to respond, so I will list the quotes and then respond.
            Act: list_quotes()
            Question: Who is the current president of the US?
            Query: current US president
            - Snippet 1: "Joe Biden: The President | White House"
            - Snippet 3: "The incumbent president is Joe Biden"
            Thought: I will write a response now.
            Act: respond(The current president of the US is Joe Biden)
            ---
            User: What is the capital of France?
            Thought: It is really unlikely that the capital of France changed since my database was last updated. So, I don't need to search the web, and can respond.
            Act: respond(The capital of France is Paris)
            ---
            User: What is the capital of Georgia?
            Thought: The user can be asking about Georgia the country or Georgia the state. I should confirm which one before issuing a final response
            Act: respond(Do you mean the state or the country?)
            Observation:
            User: The state
            Thought: The capital of Georgia probably didn't change since my database was last updated, so I can respond. 
            Act: respond(The capital of Georgia is Atlanta)
            ---
            User: Who is richer, Messi or Ronaldinho Gaucho?
            Thought: To answer this, I will need to search for each of their net worths, and then compare.
            Act: search(Ronaldinho Gaucho net worth)
            Observation:
            [1] Ronaldinho&#39;s Net Worth (Updated 2023) | Wealthy Gorilla
            Introduction As of September 2023, Ronaldinho’s net worth is estimated to be roughly $90 Million. Ronaldo de Assis Moreira, better known as Ronaldinho, is a Brazilian former professional footballer and ambassador for Barcelona. from Porto Alegre. He is regarded to be one of the best football players of all time. Early Life

            [2] Ronaldinho Net Worth | Celebrity Net Worth
            Ronaldinho is a retired Brazilian football (soccer) player who has a net worth of $90 million. Ronaldinho was once considered to be the best soccer player in the world. He was twice the...

            [3] Ronaldinho Net Worth: 2022, Career, House, Cars &amp; Lifestyle - Players Bio
            A Brazilian legend, Ronaldinho has an outstanding net worth of $90 million. Even at his time, this sum was as impressive as it is now. Ronaldinho is $90 million rich because of football. But let me stop you right there. It’s not in everybody’s forte to excel as the Brazilian did.
            
            Thought: All snippets seem to talk about his net worth.
            Act: extract_quotes()
            Question: Who is richer, Messi or Ronaldinho Gaucho?
            Relevant quotes from the snippets:
            - Snippet 1: "Ronaldinho’s net worth is estimated to be roughly $90 Million"
            - Snippet 2: "Ronaldinho is a retired Brazilian football (soccer) player who has a net worth of $90 million."
            - Snippet 3: "Ronaldinho has an outstanding net worth of $90 million"
            Thought: I still need to search for Messi's net worth and compare
            Act: search(messi net worth)
            Observation:
            [1] Lionel Messi - Forbes
            $130M 2023 The World's Highest-Paid Athletes Earnings as of 5/16/23 Photo by Antonio Borga/Eurasia Sport Images/Getty Images About Lionel Messi Messi claimed the Ballon d'Or as the world's best...

            [2] Lionel Messi Net Worth: How the Soccer Star Makes and Spends ... - Insider
            CHANDAN KHANNA/Getty Images Messi took home a mind-boggling $65 million in on-field earnings over the last year — more than all but one soccer player on the planet. His Inter Miami salary alone is worth $20,446,667 annually, per AP News, which means he earns roughly $393,000 per week or $56,000 per day from the Major League Soccer club.

            [3] Lionel Messi Net Worth
            $600 Million Salary: $75 Million Date of Birth: Jun 24, 1987 (36 years old) Place of Birth: Rosario, Argentina Gender: Male Height: 5 ft 6 in (1.69 m) Profession: Football player Nationality:...

            Thought: Snippet 1 says $130M, but it may be referring to other athletes based on the rest of the snippet.
            Snippets 2 and 3 talk about his salary (with contradictory numbers), but not his net worth. Thus, there are no relevant quotes here, and I should make another search.
            I will make the search narrower using quotes.
            Act: search(messi "net worth")
            Observation:
            [1] Lionel Messi - Forbes
            About Lionel Messi. Messi claimed the Ballon d'Or as the world's best soccer player for a record seventh time in 2021, putting him two ahead of his rival Cristiano Ronaldo. He transferred to Paris ...

            [2] Lionel Messi Net Worth: How the Soccer Star Makes and Spends ... - Insider
            His net worth is an eye-watering $600 million. Leo Messi of Paris Saint-Germain looks on during warmup before the Ligue 1 Uber Eats match between Paris Saint Germain and ESTAC Troyes at Parc des Princes on May 08, 2022 in Paris, France. Photo by Aurelien Meunier - PSG/PSG via Getty Images That's according to Celebrity Net Worth. Advertisement

            [3] Lionel Messi&#39;s new $650,000 Bentley shows of his massive net worth
            In 2023, Messi's net worth is a staggering $620 million, securing his place as the third-highest-paid athlete globally. A recent addition to Messi's extravagant lifestyle is a sleek Bentley...

            Thought: Snippets 2 and 3 list his net worth.
            Act: extract_quotes()
            Question: Who is richer, Messi or Ronaldinho Gaucho?
            Relevant quotes from the snippets:
            - Snippet 2: "His net worth is an eye-watering $600 million"
            - Snippet 3: "In 2023, Messi's net worth is a staggering $620 million"
            Thought: I have all the information I need to respond, so I'll list the quotes and then respond.
            Act: list_quotes()
            Question: Who is richer, Messi or Ronaldinho Gaucho?
            Query: Ronaldinho Gaucho net worth
            - Snippet 1: "Ronaldinho’s net worth is estimated to be roughly $90 Million"
            - Snippet 2: "Ronaldinho is a retired Brazilian football (soccer) player who has a net worth of $90 million."
            - Snippet 3: "Ronaldinho has an outstanding net worth of $90 million"
            Query: messi "net worth"
            - Snippet 2: "His net worth is an eye-watering $600 million"
            - Snippet 3: "In 2023, Messi's net worth is a staggering $620 million"
            Thought: I will write a response now.
            Act: respond(Messi has an estimated net worth between $600 and $620 million, while Ronaldinho has an estimated net worth of $90 million. Thus, Messi is richer)
            '''
    return lm

Now let's define the functions that search and extract quotes:

In [8]:
@guidance
def search(lm, query):
    # Setting this for later use
    lm = lm.set('query', query)
    # This is where search actually gets called
    lm = lm.set('snippets', format_snippets(top_snippets(query)))
    lm += '\nObservation:\n' + lm['snippets']
    return lm

@guidance
def extract_quotes(lm):
    query = lm['query']
    snippets = lm['snippets'].split('\n\n')[:-1]
    snippet_substrings = [substring(x) for x in snippets]
    # By default we have 3 snippets. The model can pick (1) which snippets it's going to quote, and then (2) a substring of that snippet
    snippet = '- Snippet ' + select([
        '1: "' + snippet_substrings[0] + '"',
        '2: "' + snippet_substrings[1] + '"',
        '3: "' + snippet_substrings[2] + '"',]) + '\n'
    # We can do one or more quotes
    lm += capture(one_or_more(snippet), name='temp_gen')
    # We save all of the quotes so that we can print them at the end
    current_quotes = lm.get('current_quotes', '')
    current_quotes += f'''Query: {query}\n{lm['temp_gen']}'''
    lm = lm.set('current_quotes', current_quotes)
    return lm

Here is an example of search, and how it adds snippets to an lm:

In [14]:
lm = llama2 + search('current use president')

And here is an example of extract_quotes:

In [15]:
lm + 'Now I will extract quotes relevant to the question "What does the president do?"\n'+  extract_quotes()

Now let's write functions to start / restart the chat, and to do the react loop:

In [16]:
@guidance
def chat_search(lm, query):
    with user():
        lm += f'User: {query}'
        lm = lm.set('user_query', query)
    with assistant():
        lm += react_loop()
    return lm

@guidance
def react_loop(lm):
    while True:
        lm += 'Thought: ' + gen('thought', list_append=True, stop='Act:')
        lm += 'Act: ' + select(['search', 'respond', 'extract_quotes', 'list_quotes'], name='action')
        action = lm['action']
        if action == 'search':
            # generate the search query
            lm += f'''({gen(name='arg', stop=')')})\n'''
            # search the web and paste result into the prompt
            lm += search(lm['arg'])
        if action == 'respond':
            # generate the response and stop the loop
            lm += f'''({gen(name='arg', stop=')')})\n'''
            break
        if action == 'extract_quotes':
            lm += '()\n'
            lm += 'Question: ' + lm['user_query'] + '\n'
            lm += 'Relevant quotes from the snippets:\n'
            lm += extract_quotes()
        if action == 'list_quotes':
            lm += '()\n'
            lm += 'Question: ' + lm['user_query'] + '\n'
            # Just paste all of the quotes we gathered throughout the loop
            lm += lm['current_quotes']
    return lm

In [17]:
# Pre-load the prompt and system message so we can reuse this
search_lm = llama2 + init_system()

Let's try a few prompts:

In [18]:
lm = search_lm + chat_search('Where does Sam Altman work?')

We didn't prompt the model with follow up questions, but let's give it a shot anyway:

In [19]:
lm += chat_search('Why did he leave OpenAI?')

Let's try a few other questions:

In [21]:
query = "What new discoveries from the James Webb Space Telescope can I tell my 9 year old about?" 
lm = search_lm + chat_search(query)

In [22]:
query = "What is 2 + 5?"
lm = search_lm + chat_search(query)

In [23]:
query = "What is the capital of Brazil?"
lm = search_lm + chat_search(query)

In [24]:
query = "Who created the guidance library?"
lm = search_lm + chat_search(query)